In [ ]:
# -------------------------- LIBRARIES NECESSARY IN THIS PROJECT  -------------------------- #
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# -------------------------- FUNCTIONS -------------------------- #

def getDataSet(data_path):
    return pd.read_csv(data_path)

def plotNaNValuesColums(dataSet):
    # Gets the dataSet Columns with null values
    #  and the Percentual of null values per column    
    missing_values = (dataSet.isnull().sum() / len(dataSet)) * 100    
    missing_values = missing_values[missing_values > 0]
    missing_values.sort_values(inplace=True)   

    missing_values = missing_values.to_frame()
    missing_values.columns = ['Percentual']
    missing_values.index.names = ['Name']
    missing_values['Column'] = missing_values.index

    # Plots the graph
    sns.set(style="whitegrid", color_codes=True)
    sns.barplot(x = 'Column', y = 'Percentual', data=missing_values)
    plt.xticks(rotation = 90)
    plt.show()
    print(missing_values)

def plotNotNaNValuesColums(dataSet):
    # Gets the dataSet Columns with null values
    #  and the Percentual of null values per column    
    missing_values = (dataSet.notnull().sum() / len(dataSet)) * 100    
    missing_values = missing_values[missing_values > 0]
    missing_values.sort_values(inplace=True)   

    missing_values = missing_values.to_frame()
    missing_values.columns = ['Percentual']
    missing_values.index.names = ['Name']
    missing_values['Column'] = missing_values.index

    # Plots the graph
    sns.set(style="whitegrid", color_codes=True)
    sns.barplot(x = 'Column', y = 'Percentual', data=missing_values)
    plt.xticks(rotation = 90)
    plt.show()
    print(missing_values)

    

# -------------------------- PATH OF THE DATASETS USED IN THE PROJECT  -------------------------- #

invoices_claims_last_actived_path="dataSets/Invoices_Claims_Last_Actived.csv"
invoices_claims_last_completed_path="dataSets/Invoices_Claims_Last_Completed.csv"
invoices_claims_path="dataSets/Invoices_Claims.csv"
members_supported="dataSets/Members_Supported.csv"
plans_budgets="dataSets/Plans_Budgets.csv"
providers="dataSets/Providers.csv"

## Upload dataSets

In [ ]:
# upload the dataSet with invoices and claims data
invoices_claims_last_actived_all_fields = getDataSet(invoices_claims_last_actived_path)
invoices_claims_last_completed_all_fields = getDataSet(invoices_claims_last_completed_path)

# upload the dataSet with invoices and claims data
df_plans_budgets_all_fields = getDataSet(plans_budgets)

# upload the dataSet with invoices and claims data
df_members_supported_all_fields = getDataSet(members_supported)

## Data manipulation and cleansing<a name="preparation"></a>

### Members dataset cleasing

In [ ]:
# Check the dataSet fields
df_members_supported_all_fields.info()

In [ ]:
df_members_supported_all_fields.head(4)

In [ ]:
# Select just the fields to be used
df_members_supported = df_members_supported_all_fields[{"id"
                                                      , "member_key" 
                                                      , "first_name"
                                                      , "last_name"
                                                      , "price_zone_code"
                                                      , "u_disabilities"
                                                      , "SA1"
                                                      }]

# Rename some dataSet colums to create a name's pattern
df_members_supported.rename(columns={'id':'member_id'
                                   , 'u_disabilities':'disabilities'
                                    } , inplace = True)                                    

In [ ]:
# Check if there any NaN Field
plotNaNValuesColums(df_members_supported)

# Although amount of null values in the columns is high they are keeped in the dataSet
#  And it is assigned 'No Assigned' to the null values in u_disabilities and SA1
df_members_supported.loc[df_members_supported['disabilities'].isnull(),['disabilities']] = "No Assigned"
df_members_supported.loc[df_members_supported['SA1'].isnull(),['SA1']] = "No Assigned"

df_members_cleased = df_members_supported

### Plans and budgets dataset cleasing

In [ ]:
# Check the dataSet fields
df_plans_budgets_all_fields.info()

In [ ]:
df_plans_budgets_all_fields.head(4)

In [ ]:
# Select just the fields to be used
df_plans_budgets = df_plans_budgets_all_fields[{"member_key"
                                              , "plan_key"
                                              , "plan_start_date" 
                                              , "plan_start_date.1"
                                              , "status"
                                              , "budget_number"
                                              , "level2_key"
                                              , "level2_name"
                                              , "level1_key"
                                              , "level1_name"
                                              , "item_category_level3_key"
                                              , "name_stated_item"
                                              , "opening_balance"
                                              , "closing_balance"
                                              , "value_allocated_budget"
                                              , "status_budget"
                                              }]


# Rename some dataSet colums to create a name's pattern
df_plans_budgets.rename(columns={'plan_start_date.1':'plan_end_date'
                               , 'status':'plan_status'                               
                               , 'status_budget':'budget_status'
                               , 'value_allocated_budget':'budget_value_allocated'
                               , 'level2_key':'budget_level2_key'
                               , 'level2_name':'budget_level2_name'
                               , 'level2_display_name':'budget_level2_display_name'
                               , 'level1_key':'budget_level1_key'
                               , 'level1_name':'budget_level1_name'
                               , 'item_category_level3_key':'budget_level3_key'
                               , 'level3_reference_number':'budget_level3_reference_number'
                               , 'name_stated_item':'budget_level3_name'                               
                                } , inplace = True)                            

In [ ]:
# Check the Budget's status in the dataSet
np.unique(df_plans_budgets['budget_status'])

In [ ]:
# Check if there any Nan Field
print(plotNaNValuesColums(df_plans_budgets))

In [ ]:
# Check some budget_number NaN Data to
#  Check these data have in commum
#   It is seen that when budget_number is null
#    opening_balance and closing_balance are not null
Null_budget_number = df_plans_budgets.loc[(df_plans_budgets['budget_number'].isna())]
Null_budget_number.head(4)

In [ ]:
# Check some budget_number Not NaN Data to
#  Check these data have in commum
#   It is seen that when budget_number is null
#    opening_balance and closing_balance are null
Null_budget_number = df_plans_budgets.loc[(df_plans_budgets['budget_number'].notna())]
Null_budget_number.head(4)

In [ ]:
# Plot the percentual of opening_balance and closing_balance not Nan with budget_number Nan
Null_budget_number = df_plans_budgets[{'opening_balance', 'closing_balance'}].loc[(df_plans_budgets['budget_number'].isna())]
plotNotNaNValuesColums(Null_budget_number)

# Deletes the records with budget_number are NaN
#  As They are rows that represents Balance
df_plans_budgets = df_plans_budgets.drop(df_plans_budgets[df_plans_budgets.budget_number.isna()].index)

#  And keeps the ones with level3_reference_number, stated_item_name, level3_key
#   Assigns 'no assigned' in columns 'level3_key','level3_reference_number', 'stated_item_name'
#   because if level3_reference_number, stated_item_name, level3_key are NaN
#   Means the budget starts in level 2
df_plans_budgets.loc[df_plans_budgets['budget_level3_key'].isnull(),['budget_level3_key', 'budget_level3_name']] = "No Assigned"

In [ ]:
# Check if there is any DateTime column
df_plans_budgets.select_dtypes(include=[np.datetime64]).any().count()

In [ ]:
# Group the plans from their bugdets
df_plans_budgets_cleased = df_plans_budgets.groupby(["member_key"
                                                   , "plan_key"
                                                   , "plan_status"
                                                   , "plan_start_date"
                                                   , "plan_end_date"
                                                   , "budget_status"
                                                   , "budget_level3_name"
                                                   , "budget_level3_key" 
                                                   , "budget_level2_key"
                                                   , "budget_level2_name"
                                                   , "budget_level1_key"
                                                   , "budget_level1_name"]).agg({"budget_value_allocated": "sum"}).reset_index()                                                   

### Invoice and claims dataset cleasing

### Invoice and claims dataset From LAST ACTIVED plan

In [ ]:
# Check the dataSet fields
invoices_claims_last_actived_all_fields.info()

In [ ]:
invoices_claims_last_actived_all_fields.head(4)

In [ ]:
# Select just the fields to be used
invoices_claims_last_actived = invoices_claims_last_actived_all_fields[{"member_id"
                                                                      , "invoice_state"
                                                                      , "invoice_id"
                                                                      , "claim_state"
                                                                      , "claim_id"
                                                                      , "key"  
                                                                      , "key.1"
                                                                      , "key.2"
                                                                      , "claim_start_date"
                                                                      , "claim_end_date"
                                                                      , "claim_funded_amount"
                                                                      , "invoice_total"
                                                                      , "claimed_units"
                                                                      , "claimed_unit_price"
                                                                      }]

# Rename some dataSet colums to create a name's pattern
invoices_claims_last_actived.rename(columns={'claim_funded_amount':'funded_amount'
                                           , 'invoice_total':'invoiced_amount_total'
                                           , 'claimed__units_price':'claimed__units_amount'  
                                           , 'key.1':'claim_level2_key'
                                           , 'key.2':'claim_level1_key'
                                           , 'key':'claim_level3_key'
                                              } , inplace = True)                  

In [ ]:
# Check the claim_state To check whether there is any dump value
invoices_claims_last_actived['claim_state'].unique()

In [ ]:
invoices_claims_last_actived = invoices_claims_last_actived.drop(invoices_claims_last_actived[invoices_claims_last_actived.invoice_state.isna()].index)

In [ ]:
# Check the claim_state To check whether there is any dump value
invoices_claims_last_actived['invoice_state'].unique()

In [ ]:
invoices_claims_last_actived['claim_state'].unique()

In [ ]:
# Check if there any Nan Field
print(plotNaNValuesColums(invoices_claims_last_actived))

In [ ]:
# Check some invoiced_amount and invoiced_amount NaN Data to
#  Check these data have in commum
df_invoiced_units_NaN = invoices_claims_last_actived.loc[invoices_claims_last_actived['funded_amount'].isna()]
df_invoiced_units_NaN

In [ ]:
# Group the state of invoice and claim of all null data
#  And Keeps these records
#   Because if 'funded_amount' is NaN means the claims is not PAID OR REFUNDED
df_invoiced_units_NaN[{"invoice_state"
                     , "claim_state"
                      }].groupby(["invoice_state"
                                , "claim_state"
                                 ]).count()

In [ ]:
# As seeing above some invoices stated ALL_PAID and PAID are also funded_amount NaN
# Check The NaN fields
len(invoices_claims_last_actived[(invoices_claims_last_actived['funded_amount'].isna())
                               & (invoices_claims_last_actived['invoice_state'] == 'ALL_PAID')
                               & (invoices_claims_last_actived['claim_state'] == 'PAID')
                                ]
   )

In [ ]:
df_null_funded_amount =  invoices_claims_last_actived[(invoices_claims_last_actived['funded_amount'].isna())
                                                    & (invoices_claims_last_actived['invoice_state'] == 'ALL_PAID')
                                                    & (invoices_claims_last_actived['claim_state'] == 'PAID')
                                                     ]

# Fixes The funded_amount NaN in invoice_state = ALL_PAID and claim_state = PAID
df_null_funded_amount['funded_amount'] = df_null_funded_amount['claimed_unit_price'].astype(float) * df_null_funded_amount['claimed_units'].astype(float)


invoices_claims_last_actived = invoices_claims_last_actived.set_index('claim_id')
invoices_claims_last_actived.update(df_null_funded_amount.set_index('claim_id'))
invoices_claims_last_actived.reset_index(inplace=True)                                   

In [ ]:
# Group the invoices from their claims
invoices_claims_last_actived_cleased = invoices_claims_last_actived.groupby(["member_id"
                                                                           , "invoice_state" 
                                                                           , "claim_state"
                                                                           , "claim_level1_key"
                                                                           , "claim_level2_key"
                                                                           , "claim_level3_key"
                                                                           , "invoiced_amount_total"]).agg({"funded_amount": "sum"}).reset_index()

### Invoice and claims dataset From LAST COMPLETED plan

In [ ]:
# Check the dataSet fields
invoices_claims_last_completed_all_fields.info()

In [ ]:
invoices_claims_last_completed_all_fields.head(4)

In [ ]:
# Select just the fields to be used
invoices_claims_last_completed = invoices_claims_last_completed_all_fields[{"member_id"
                                                                          , "invoice_state"
                                                                          , "invoice_id"
                                                                          , "claim_state"
                                                                          , "claim_id"
                                                                          , "key"  
                                                                          , "key.1"
                                                                          , "key.2"
                                                                          , "claim_start_date"
                                                                          , "claim_end_date"
                                                                          , "claim_funded_amount"
                                                                          , "invoice_total"
                                                                          , "claimed_units"
                                                                          , "claimed_unit_price"
                                                                      }]

# Rename some dataSet colums to create a name's pattern
invoices_claims_last_completed.rename(columns={'claim_funded_amount':'funded_amount'
                                             , 'invoice_total':'invoiced_amount_total'
                                             , 'claimed__units_price':'claimed__units_amount'  
                                             , 'key.1':'claim_level2_key'
                                             , 'key.2':'claim_level1_key'
                                             , 'key':'claim_level3_key'
                                              } , inplace = True) 

In [ ]:
# Check the claim_state To check whether there is any dump value
invoices_claims_last_completed['claim_state'].unique()

In [ ]:
# Check the claim_state To check whether there is any dump value
invoices_claims_last_completed['invoice_state'].unique()

In [ ]:
# Check if there any Nan Field
print(plotNaNValuesColums(invoices_claims_last_completed))

In [ ]:
# Check some invoiced_amount and invoiced_amount NaN Data to
#  Check these data have in commum
df_invoiced_units_NaN = invoices_claims_last_completed.loc[invoices_claims_last_completed['funded_amount'].isna()]
df_invoiced_units_NaN

In [ ]:
df_null_funded_amount =  invoices_claims_last_completed[(invoices_claims_last_completed['funded_amount'].isna())
                                                      & (invoices_claims_last_completed['invoice_state'] == 'ALL_PAID')
                                                      & (invoices_claims_last_completed['claim_state'] == 'PAID')
                                                     ]

# Fixes The funded_amount NaN in invoice_state = ALL_PAID and claim_state = PAID
df_null_funded_amount['funded_amount'] = df_null_funded_amount['claimed_unit_price'].astype(float) * df_null_funded_amount['claimed_units'].astype(float)


invoices_claims_last_completed = invoices_claims_last_completed.set_index('claim_id')
invoices_claims_last_completed.update(df_null_funded_amount.set_index('claim_id'))
invoices_claims_last_completed.reset_index(inplace=True)    

In [ ]:
# Group the state of invoice and claim of all null data
#  And Keeps these records
#   Because if 'funded_amount' is NaN means the claims is not PAID OR REFUNDED
df_invoiced_units_NaN[{"invoice_state"
                     , "claim_state"
                      }].groupby(["invoice_state"
                                , "claim_state"
                                 ]).count()

In [ ]:
# As seeing above some invoices stated ALL_PAID and PAID are also funded_amount NaN
# Check The NaN fields
len(invoices_claims_last_completed[(invoices_claims_last_completed['funded_amount'].isna())
                                 & (invoices_claims_last_completed['invoice_state'] == 'ALL_PAID')
                                 & (invoices_claims_last_completed['claim_state'] == 'PAID')
                                ]
   )

In [ ]:
# Group the invoices from their claims
invoices_claims_last_completed_cleased = invoices_claims_last_completed.groupby(["member_id"
                                                                               , "invoice_state" 
                                                                               , "claim_state"
                                                                               , "claim_level1_key"
                                                                               , "claim_level2_key"
                                                                               , "claim_level3_key"
                                                                               , "invoiced_amount_total"]).agg({"funded_amount": "sum"}).reset_index()

### Merge dataSets cleased

### Merges members dataSet with plans and budgets dataSet

In [ ]:
# Member dataSet with plans and budgets dataSet
df_members_plans_budgets_merged = pd.merge(df_members_cleased, df_plans_budgets_cleased, on=["member_key", "member_key"])

### Merges last PLAN_DELIVERY_ACTIVED plan with members plans budgets merged dataSet

In [ ]:
invoices_claims_plan_delivery_active = df_members_plans_budgets_merged.loc[(df_members_plans_budgets_merged['plan_status'] == 'PLAN_DELIVERY_ACTIVE')]

invoices_claims_plan_delivery_active = invoices_claims_plan_delivery_active.merge(invoices_claims_last_actived_cleased, on='member_id')

invoices_claims_plan_delivery_active

In [ ]:
invoices_claims_plan_delivery_active = invoices_claims_plan_delivery_active[

   (
      (invoices_claims_plan_delivery_active.budget_level3_key == 'No Assigned')
    & (invoices_claims_plan_delivery_active.budget_level2_key == invoices_claims_plan_delivery_active.claim_level2_key)
    
   )

   |

   (
      (invoices_claims_plan_delivery_active.budget_level3_key != 'No Assigned')
    & (invoices_claims_plan_delivery_active.budget_level3_key == invoices_claims_plan_delivery_active.claim_level3_key)
    & (invoices_claims_plan_delivery_active.budget_level2_key == invoices_claims_plan_delivery_active.claim_level2_key)
    & (invoices_claims_plan_delivery_active.budget_level1_key == invoices_claims_plan_delivery_active.budget_level1_key)
   )

       
]

### Merges last COMPLETED plan with members plans budgets merged dataSet

In [ ]:
invoices_claims_plan_completed = df_members_plans_budgets_merged.loc[(df_members_plans_budgets_merged['plan_status'] == 'COMPLETED')]

invoices_claims_plan_completed = invoices_claims_plan_completed.merge(invoices_claims_last_completed_cleased, on='member_id')

invoices_claims_plan_completed = invoices_claims_plan_completed[

   (
      (invoices_claims_plan_completed.budget_level3_key == 'No Assigned')
    & (invoices_claims_plan_completed.budget_level2_key == invoices_claims_plan_completed.claim_level2_key)
    
   )

   |

   (
      (invoices_claims_plan_completed.budget_level3_key != 'No Assigned')
    & (invoices_claims_plan_completed.budget_level3_key == invoices_claims_plan_completed.claim_level3_key)
    & (invoices_claims_plan_completed.budget_level2_key == invoices_claims_plan_completed.claim_level2_key)
    & (invoices_claims_plan_completed.budget_level1_key == invoices_claims_plan_completed.budget_level1_key)
   )
        
]

### Merges last COMPLETED plan dataSet with PLAN_DELIVERY_ACTIVED plan

In [ ]:
frames = [invoices_claims_plan_delivery_active, invoices_claims_plan_completed]
invoices_claims_merged = pd.concat(frames)

In [ ]:
df_model = invoices_claims_merged.groupby(["member_key"
                                         , "first_name"
                                         , "last_name"
                                         , "disabilities"
                                         , "SA1"
                                         , "price_zone_code"                                         
                                         , "plan_key"
                                         , "plan_status"
                                         , "plan_start_date"
                                         , "plan_end_date"                                                                               
                                         , "budget_level3_name"
                                         , "budget_level3_key"
                                         , "budget_level2_key"
                                         , "budget_level2_name"
                                         , "budget_level1_key"
                                         , "budget_level1_name"
                                         , "budget_value_allocated"  	
                                         , "invoice_state"
                                         , "claim_state"
                                          ]).agg({"invoiced_amount_total": "sum", "funded_amount": "sum"}).reset_index()                                                          

In [ ]:
df_model.loc[(df_model['member_key'] == '007c1090-d6ca-11ea-bb8a-a73bf14c75f9')
           & (df_model['budget_level2_key'] == '7d0af57b-f597-11e9-bfb4-022d4762bb3c')         
]  